In [1]:
# Install package
!pip install whoosh
!pip install pytrec_eval
!pip install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308216 sha256=6366212819318e56d61782233d84195ea076d4c179a3eb4d791b2c5a8c83073f
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=bcaea1715542c655fbcd58216cd32ea1dcb417abc641f21a94e04a4763bf55f2
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
# Import package
import wget
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import wget
import abc
from abc import abstractmethod
from whoosh.analysis import Filter
import nltk
from nltk.stem import *
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from whoosh import scoring


# Download required resources
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!unzip data_zero.zip

Archive:  data_zero.zip
   creating: data_zero/
  inflating: data_zero/.DS_Store     
  inflating: __MACOSX/data_zero/._.DS_Store  
  inflating: data_zero/data.topics   
  inflating: __MACOSX/data_zero/._data.topics  
  inflating: data_zero/data.qrels    
  inflating: __MACOSX/data_zero/._data.qrels  
   creating: data_zero/documents/
  inflating: data_zero/documents/.DS_Store  
  inflating: __MACOSX/data_zero/documents/._.DS_Store  
   creating: data_zero/documents/00/
  inflating: data_zero/documents/00/.DS_Store  
  inflating: __MACOSX/data_zero/documents/00/._.DS_Store  
  inflating: data_zero/documents/00/G00-00-0000007  
  inflating: __MACOSX/data_zero/documents/00/._G00-00-0000007  
  inflating: data_zero/documents/00/G00-00-0000009  
  inflating: __MACOSX/data_zero/documents/00/._G00-00-0000009  
  inflating: data_zero/documents/00/G00-00-0000008  
  inflating: __MACOSX/data_zero/documents/00/._G00-00-0000008  
  inflating: data_zero/documents/00/G00-00-0000006  
  inflating: _

In [4]:
!unzip data_few.zip

Archive:  data_few.zip
   creating: data_few/
  inflating: data_few/.DS_Store      
  inflating: __MACOSX/data_few/._.DS_Store  
  inflating: data_few/data.topics    
  inflating: __MACOSX/data_few/._data.topics  
  inflating: data_few/data.qrels     
  inflating: __MACOSX/data_few/._data.qrels  
   creating: data_few/documents/
  inflating: data_few/documents/.DS_Store  
  inflating: __MACOSX/data_few/documents/._.DS_Store  
   creating: data_few/documents/00/
  inflating: data_few/documents/00/.DS_Store  
  inflating: __MACOSX/data_few/documents/00/._.DS_Store  
  inflating: data_few/documents/00/G00-00-0000007  
  inflating: __MACOSX/data_few/documents/00/._G00-00-0000007  
  inflating: data_few/documents/00/G00-00-0000009  
  inflating: __MACOSX/data_few/documents/00/._G00-00-0000009  
  inflating: data_few/documents/00/G00-00-0000008  
  inflating: __MACOSX/data_few/documents/00/._G00-00-0000008  
  inflating: data_few/documents/00/G00-00-0000006  
  inflating: __MACOSX/data_few/d

In [5]:
!unzip data_chain.zip

Archive:  data_chain.zip
   creating: data_chain/
  inflating: data_chain/.DS_Store    
  inflating: __MACOSX/data_chain/._.DS_Store  
  inflating: data_chain/data.topics  
  inflating: __MACOSX/data_chain/._data.topics  
  inflating: data_chain/data.qrels   
  inflating: __MACOSX/data_chain/._data.qrels  
   creating: data_chain/documents/
  inflating: data_chain/documents/.DS_Store  
  inflating: __MACOSX/data_chain/documents/._.DS_Store  
   creating: data_chain/documents/00/
  inflating: data_chain/documents/00/.DS_Store  
  inflating: __MACOSX/data_chain/documents/00/._.DS_Store  
  inflating: data_chain/documents/00/G00-00-0000007  
  inflating: __MACOSX/data_chain/documents/00/._G00-00-0000007  
  inflating: data_chain/documents/00/G00-00-0000009  
  inflating: __MACOSX/data_chain/documents/00/._G00-00-0000009  
  inflating: data_chain/documents/00/G00-00-0000008  
  inflating: __MACOSX/data_chain/documents/00/._G00-00-0000008  
  inflating: data_chain/documents/00/G00-00-000000

In [6]:
class IRSystem(metaclass=abc.ABCMeta):
    """
    Abstract class which is inherited by other IR system
    """

    def __init__(self, data_dir):
        # DON'T change the following definitions for topic_file, qrels_file, document_dir, file_list
        self.topic_file = os.path.join(data_dir, "data.topics")
        self.qrels_file = os.path.join(data_dir, "data.qrels")
        self.document_dir = os.path.join(data_dir, "documents")
        self.file_list = [str(filePath) for filePath in Path(self.document_dir).glob("**/*") if filePath.is_file()]

        self.create_index()
        self.create_parser_searcher()

    @abstractmethod
    def create_index(self):
        pass

    @abstractmethod
    def add_files(self):
        pass

    @abstractmethod
    def create_parser_searcher(self):
        pass

    @abstractmethod
    def perform_search(self, topic_phrase):
        pass

    @staticmethod
    def post_process_score(score):
        return score

    @staticmethod
    def print_trec_eval_result(results):

        if not results:
            print('empty results')
            return

        def print_line(name, scope, num):
            print('{:25s}{:8s}{:.4f}'.format(name, scope, num))

        for query_id, query_measures in results.items():
            for measure, value in query_measures.items():
                if measure == "runid":
                    continue
                print_line(measure, query_id, value)

        for measure in query_measures.keys():
            if measure == "runid":
                continue
            print_line(
                measure,
                'all',
                pytrec_eval.compute_aggregated_measure(
                    measure,
                    [query_measures[measure]
                     for query_measures in results.values()]))


    def score(self,docnum,topic_results, topic_phrase):
        return topic_results.score(docnum)


    def print_rel_name(self, q_id):
        with open(self.topic_file, "r") as tf:
            topics = tf.read().splitlines()
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            if topic_id == q_id:
                print("---------------------------Topic_id and Topic_phrase----------------------------------")
                print(topic_id, topic_phrase)
                 # get search result
                topic_results = self.perform_search(topic_phrase)
                print("---------------------------Return documents----------------------------------")
                for (docnum, result) in enumerate(topic_results):
                    score = self.score(docnum, topic_results, topic_phrase)
                    score = self.post_process_score(score)
                    print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                print("---------------------------Relevant documents----------------------------------")
                with open(self.qrels_file, 'r') as f_qrel:
                    qrels = f_qrel.readlines()
                    for i in qrels:
                        qid, _, doc, rel = i.rstrip().split(" ")
                        if qid == q_id and rel == "1":
                            print(i.rstrip())

    def py_trec_eval(self):

        self.create_parser_searcher()
        # Load topic file - a list of topics(search phrases) used for evalutation
        with open(self.topic_file, "r") as tf:
            topics = tf.read().splitlines()

            # create an output file to which we'll write our results
        temp_output_file = tempfile.mkstemp()[1]
        with open(temp_output_file, "w") as outputTRECFile:
            # for each evaluated topic:
            # build a query and record the results in the file in TREC_EVAL format
            for topic in topics:
                topic_id, topic_phrase = tuple(topic.split(" ", 1))
                # get search result
                topic_results = self.perform_search(topic_phrase)
                # format the result
                for (docnum, result) in enumerate(topic_results):
                    score = self.score(docnum, topic_results, topic_phrase)
                    outputTRECFile.write(
                        "%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                    topic_with_result = topic_id


        with open(self.qrels_file, 'r') as f_qrel:
            qrel = pytrec_eval.parse_qrel(f_qrel)

        with open(temp_output_file, 'r') as f_run:
            run = pytrec_eval.parse_run(f_run)

        evaluator = pytrec_eval.RelevanceEvaluator(
            qrel, pytrec_eval.supported_measures)

        results = evaluator.evaluate(run)

        #fill results dictionary with queries that were returned 0 documents
        topic_ids = {t.split()[0] for t in topics}
        for emptyresult_topicid in topic_ids.difference(set(results.keys())):
            num_rel = float(sum(qrel[emptyresult_topicid].values()))
            if num_rel>0:
              topic_stats={measure:0.0 for measure in results[topic_with_result]}
            else:
              topic_stats={measure:1.0 for measure in results[topic_with_result]}
            topic_stats["num_rel"]=num_rel
            topic_stats["num_ret"] = 0.0
            topic_stats["num_rel_ret"] = 0.0
            topic_stats["num_q"]=1.0

            results[emptyresult_topicid] = topic_stats


        self.print_trec_eval_result(results)

In [7]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

# Dont change this! Use it as-is in your code if you rerank your results using a non-Whoosh scoring function
class NeuralResults():
  '''
  This class is used to rerank documents returned by whoosh in an interface that
  imitates whoosh.searching.Results (the datatype of topicResults in pyTrecEval)
  '''
  def __init__(self, booleansearchdocs,scores,rankings, file_list):
    self.results=[]
    if rankings.shape:
      for idx in rankings:
        self.results.append({'file_path':file_list[booleansearchdocs[idx]],'score':scores[idx] })

  def score(self,docnum):
    return self.results[docnum]['score']

  def __iter__(self):
    return self.results.__iter__()

In [8]:
class IR_zero(IRSystem):
    def create_index(self):
        """
        INPUT:
            None
        OUTPUT:
            None

        NOTE: Please update self.index_sys which should have type whoosh.index.FileIndex
        """
        # DON't change the name of 'index_sys'
        self.index_sys = None
        # Generate a temporary directory for the index
        indexDir = tempfile.mkdtemp()
        # Define a Schema for the index
        mySchema = Schema(file_path=ID(stored=True),
                          file_content= \
                          TEXT(analyzer=RegexTokenizer() | LowercaseFilter() \
                               | IntraWordFilter() | StopFilter() | StemFilter()))
        # Create and return the index
        self.index_sys = index.create_in(indexDir, mySchema)


    def add_files(self):
        """
        INPUT:
            None
        OUTPUT:
            None

        NOTE: Add buffer to self.index_sys
        """
        # Open writer
        writer = writing.BufferedWriter(self.index_sys, period=None, limit=1000)
        try:
          for docNum, filePath in enumerate(self.file_list):
            with open(filePath, "r", encoding="utf-8", errors="ignore") as f:
              fileContent = f.read()
              writer.add_document(file_path = filePath,
                                  file_content = fileContent)
              # Print status every 1000 documents
              if (docNum+1) % 1000 == 0:
                print("already indexed:", docNum+1)
          print("done indexing.")
        finally:
          # Close the index
          writer.close()


    def create_parser_searcher(self):
        """
        INPUT:
            None
        OUTPUT:
            None

        NOTE: Please update self.query_parser and self.self.searcherwhich should have type whoosh.qparser.default.QueryParser and whoosh.searching.Searcher respectively
        """
        # DON'T change the names of 'query_parser' and 'searcher'
        self.query_parser = None
        self.searcher = None
        # Create the query parser for the "file_content"
        self.query_parser = QueryParser("file_content", \
                                        schema=self.index_sys.schema)
        # Initialize a searcher
        self.searcher = self.index_sys.searcher()


    def perform_search(self, topic_phrase):
        """
        INPUT:
            topic_phrase: string
        OUTPUT:
            topicResults: whoosh.searching.Results OR NeuralResults

        NOTE: Utilize self.query_parser and self.searcher to calculate the result for topic_phrase
        """
        # Remove special characters and punctuation
        topics = re.sub(r"[^a-zA-Z0-9]", " ", topic_phrase)
        # Tokenization and lowercase
        words = topics.lower().split()
        # Remove stopwords
        stop_words = set(stopwords.words("english"))
        filtered_words = [word for word in words if word not in stop_words]
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
        # Rejoin lemmatized tokens into cleaned topics
        cleaned_topics = " ".join(lemmatized_words)
        query = self.query_parser.parse(cleaned_topics)
        topicResults = self.searcher.search(query, limit=None)
        return topicResults

In [9]:
zero_shot = IR_zero("data_zero")

In [10]:
zero_shot.add_files()

done indexing.


In [11]:
zero_shot.py_trec_eval()

num_q                    4       1.0000
num_ret                  4       1.0000
num_rel                  4       1.0000
num_rel_ret              4       1.0000
map                      4       1.0000
gm_map                   4       0.0000
Rprec                    4       1.0000
bpref                    4       1.0000
recip_rank               4       1.0000
iprec_at_recall_0.00     4       1.0000
iprec_at_recall_0.10     4       1.0000
iprec_at_recall_0.20     4       1.0000
iprec_at_recall_0.30     4       1.0000
iprec_at_recall_0.40     4       1.0000
iprec_at_recall_0.50     4       1.0000
iprec_at_recall_0.60     4       1.0000
iprec_at_recall_0.70     4       1.0000
iprec_at_recall_0.80     4       1.0000
iprec_at_recall_0.90     4       1.0000
iprec_at_recall_1.00     4       1.0000
P_5                      4       0.2000
P_10                     4       0.1000
P_15                     4       0.0667
P_20                     4       0.0500
P_30                     4       0.0333


In [12]:
# Example of the desired output
# Define the queries
queries = [
    "Latest advancements in solar and wind energy technologies",
    "Impact of artificial intelligence on patient diagnosis accuracy",
    "Consequences of Arctic ice melt on global sea levels",
    "Applications of quantum computing in cryptography",
    "Effectiveness of crop rotation in sustainable farming practices",
    "Emerging trends in cybersecurity threats in the financial sector",
    "COVID-19's long-term effects on small business sustainability",
    "Influence of social media on adolescent self-esteem",
    "Comparative analysis of battery life in 2023 electric vehicle models",
    "Conservation strategies for endangered species in the Amazon rainforest"
]

# Initialize the lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# Function to process text
def process_text(text):
    # Lowercase
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stop words and punctuation, and lemmatize
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(processed_tokens)

# Process each query
processed_queries = [process_text(query) for query in queries]
print(processed_queries)

['latest advancement solar wind energy technology', 'impact artificial intelligence patient diagnosis accuracy', 'consequence arctic ice melt global sea level', 'application quantum computing cryptography', 'effectiveness crop rotation sustainable farming practice', 'emerging trend cybersecurity threat financial sector', 'effect small business sustainability', 'influence social medium adolescent', 'comparative analysis battery life electric vehicle model', 'conservation strategy endangered specie amazon rainforest']


#Prompt Engineering
Include any code used for your final prompts (e.g. API calls).


#Evaluation
Include any functions for evaluation and result processing.

In [13]:
few_shot = IR_zero("data_few")

In [14]:
few_shot.add_files()

done indexing.


In [15]:
few_shot.py_trec_eval()

num_q                    10      1.0000
num_ret                  10      1.0000
num_rel                  10      1.0000
num_rel_ret              10      1.0000
map                      10      1.0000
gm_map                   10      0.0000
Rprec                    10      1.0000
bpref                    10      1.0000
recip_rank               10      1.0000
iprec_at_recall_0.00     10      1.0000
iprec_at_recall_0.10     10      1.0000
iprec_at_recall_0.20     10      1.0000
iprec_at_recall_0.30     10      1.0000
iprec_at_recall_0.40     10      1.0000
iprec_at_recall_0.50     10      1.0000
iprec_at_recall_0.60     10      1.0000
iprec_at_recall_0.70     10      1.0000
iprec_at_recall_0.80     10      1.0000
iprec_at_recall_0.90     10      1.0000
iprec_at_recall_1.00     10      1.0000
P_5                      10      0.2000
P_10                     10      0.1000
P_15                     10      0.0667
P_20                     10      0.0500
P_30                     10      0.0333


In [16]:
chain_thought = IR_zero("data_chain")

In [17]:
chain_thought.add_files()

done indexing.


In [18]:
chain_thought.py_trec_eval()

num_q                    1       1.0000
num_ret                  1       2.0000
num_rel                  1       2.0000
num_rel_ret              1       2.0000
map                      1       1.0000
gm_map                   1       0.0000
Rprec                    1       1.0000
bpref                    1       1.0000
recip_rank               1       1.0000
iprec_at_recall_0.00     1       1.0000
iprec_at_recall_0.10     1       1.0000
iprec_at_recall_0.20     1       1.0000
iprec_at_recall_0.30     1       1.0000
iprec_at_recall_0.40     1       1.0000
iprec_at_recall_0.50     1       1.0000
iprec_at_recall_0.60     1       1.0000
iprec_at_recall_0.70     1       1.0000
iprec_at_recall_0.80     1       1.0000
iprec_at_recall_0.90     1       1.0000
iprec_at_recall_1.00     1       1.0000
P_5                      1       0.4000
P_10                     1       0.2000
P_15                     1       0.1333
P_20                     1       0.1000
P_30                     1       0.0667
